# Basic setup for fetching data from the facebook api

### Remember to add an active access token below and adjust the parameters and fields if necessary

In [1]:
# must be specified to be able to use methods of package
import sys
sys.path.append('../src') 
# import packages
from political_ads.helper import *
import pandas as pd
from political_ads.api_request import API_request
from political_ads.Preprocessor import *


In [ ]:
# insert your access token here (expires every 2 hours)
access_token = "EAADqFZC76BDMBAO2QwwFelyBTOYZB0Nig3jw6xpHVBo8OTPemZBNWVNZCxQPjFf6UdRNDNSfxsxdUkp6Of3W0vNMzyq0B12hVyv3uQkVcmQoomRL1bm02FRJt2m1N8tiA54T6fN8gPLOoV6AvtTXhbQ5nE6xZAej2c09s3BmPEv5N0kBEZBa76ArjXyWcZByaNRxE4cCmo3GdZBZAnYa0ytEZA8ZCPq5PiZAgknO5U6UcFZAR8x1MPnywIZAufQSg6ZCbUXAfYZD"
# get_new_access_token: https://developers.facebook.com/tools/explorer/


#https://www.facebook.com/ads/library/report
# "../data/generated_dataset.txt",

# PARAMETERS NECESSARY
ad_type = "POLITICAL_AND_ISSUE_ADS"
ad_reached_countries = ['US'] # Facebook delivered the ads in these countries. Provided as ISO country codes.
search_terms = "Joe Biden" # The terms to search for in your query. We treat a blank space as a logical AND and search for both terms and no other operators. The limit of your string is 100 characters or less.
limit = 1700
start_date = "2020-08-00"
end_date = "2020-11-03"

# FIELDS to specify your results
fields = ["id", "page_name", "ad_creative_body", "impressions", "delivery_by_region", "demographic_distribution", "ad_delivery_start_time", "bylines", "spend"]

# see here for more parameters and fields: https://www.facebook.com/ads/library/api/?source=archive-landing-page 

input_url = f"https://graph.facebook.com/v12.0/ads_archive?search_terms={search_terms}&ad_type={ad_type}&ad_reached_countries={ad_reached_countries}&fields={fields}&limit={limit}&ad_delivery_date_min={start_date}&ad_delivery_date_max={end_date}"

api = API_request()

response = api.get_json_response(input_url, access_token)

data = pd.DataFrame(response["data"])


In [ ]:
import time 

#https://developers.facebook.com/docs/graph-api/results

iterations = 10
sleep_time_sec = 10

counter = 0

for i in range(10):

    paging = response["paging"]["next"]
    next_response = get_json_response(paging, access_token)

    new_page = pd.DataFrame(next_response['data'])
    data = data.append(new_page, ignore_index=True)
    time.sleep(10)
    counter+=1
    print(counter)


# Looking at the data-set with all the politicians

## Making aggregate methods from their target groups (Demographic DATA)



In [2]:
#Code to read from our full dataset:
preprocess = Preprocessor()
df = preprocess.file_to_df('/home/gustavgyrst/Desktop/Research Project/political-ad-api/data/all_politicians_aggregated.txt')

In [4]:
df['demographic_distribution'].iloc[0]
df.dropna(inplace=True) #

In [5]:
# data.drop(['age_distribution', 'gender'], axis=1)

#data['age_distribution'] = data['demographic_distribution'].apply(lambda x: get_age_distribution(x, all_ages=True))

demographic_data = df[['page_name', 'demographic_distribution']].reset_index().drop(columns='index')

In [ ]:
df['demographic_distribution']
# data['impressions'] = data.impressions.apply(lambda x: transform_range(x))

In [7]:
demographic_data['gender_(M)'] = df['demographic_distribution'].apply(lambda x: get_gender_distribution(x))

In [ ]:
#method in the making
def demographic_overview(df, colname='demographic_distribution'):
    for i in range(len(df)):
        for j in range(len(df[colname][i])):
            df[colname][i][j]


#test run
df2 = demographic_data
colname = 'demographic_distribution'

#Creating the Columns
age_dist_lst = ['13-17', '18-24', '25-34', '35-44', '45-54', '55-64', '65+']
for i in age_dist_lst:
     df2[i] = 0


df2

In [ ]:
#filling in the data in the columns
for i in range(len(df2)):
    for j in range(len(df2[colname][i])):
        demo_group_per = df2[colname][i][j]
        df2[demo_group_per['age']].iloc[i] += float(demo_group_per['percentage'])

demographic_data.groupby('page_name').mean()

#data[colname][1]

#by page name 
#by gender (male, female, unknown)



In [ ]:
#https://datascience.stackexchange.com/questions/9616/how-to-create-us-state-choropleth-map
#Convert the demographic data into a heat-map



# Finding the Missing Page_names

In [3]:
df_missing = pd.read_csv('/home/gustavgyrst/Desktop/Research Project/political-ad-api/src/data_sets/missing_pid.csv')
df_all_ad_pages = pd.read_csv('/home/gustavgyrst/Desktop/Research Project/political-ad-api/data/FacebookAdLibraryReport_2021-10-15_US_lifelong_advertisers.csv')



/home/gustavgyrst/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df_missing

,Unnamed: 0,first_name,last_name,full_name,type,state,district,party,facebook,page_name,identifier,page_id,already_fetched
0,1,Maria,Cantwell,Maria Cantwell,sen,WA,NaN,Democrat,senatorcantwell,Senator Maria Cantwell,100044609161538,9351652533,missing
1,2,Benjamin,Cardin,Benjamin L. Cardin,sen,MD,NaN,Democrat,senatorbencardin,Senator Ben Cardin,100044326544838,no match,missing
2,3,Thomas,Carper,Thomas R. Carper,sen,DE,NaN,Democrat,tomcarper,Senator Tom Carper,100044593172202,no match,missing
3,4,Robert,Casey,"Robert P. Casey, Jr.",sen,PA,NaN,Democrat,SenatorBobCasey,U.S. Senator Bob Casey,100044143479624,no match,missing
4,7,Robert,Menendez,Robert Menendez,sen,NJ,NaN,Democrat,senatormenendez,Senator Bob Menendez,100044150273701,no match,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,510,Andrew,Garbarino,Andrew R. Garbarino,rep,NY,2.0,Republican,NaN,NaN,0,197834643693711,missing
177,515,Stephanie,Bice,Stephanie I. Bice,rep,OK,5.0,Republican,NaN,NaN,0,no match,missing
178,525,Blake,Moore,Blake D. Moore,rep,UT,1.0,Republican,NaN,NaN,0,no match,missing
179,532,Raphael,Warnock,Raphael G. Warnock,sen,GA,NaN,Democrat,NaN,NaN,0,no match,missing


In [5]:
df_missing[df_missing.page_id == 'no match'].count()

Unnamed: 0         166
first_name         166
last_name          166
full_name          166
type               166
state              166
district           136
party              166
facebook           132
page_name           67
identifier         166
page_id            166
already_fetched    166
dtype: int64

In [6]:
#Filtering with two values with a regex AND operation (e.g., first name "Bernie" AND "Sanders" will return all results that contain both Bernie and Sanders)
def get_page_id(data: pd.DataFrame, first_name: str, last_name:str):
    match = data[data["Page Name"].str.contains(r'^(?=.*'+first_name+')(?=.*'+last_name+')', na=False)]
    if len(match) != 0:
        return match.iloc[0][0]
    else:
        return "no match"


In [7]:
#Testing Regex filtering:
f = 'Bernie'
l = 'Sanders'
match = df_all_ad_pages[df_all_ad_pages["Page Name"].str.contains(r'^(?=.*'+f+')(?=.*'+l+')', na=False)]
match

,Page ID,Page Name,Disclaimer,Amount Spent (USD),Number of Ads in Library
26,124955570892789,Bernie Sanders,BERNIE 2020,9199592,23830
77,124955570892789,Bernie Sanders,Bernie 2020.,4132342,45749
1443,124955570892789,Bernie Sanders,Friends of Bernie Sanders,274572,825
14317,1645816202305254,The People For Bernie Sanders,Paid for by Be a Hero PAC,14463,120
44528,107229050703874,Bernie Sanders Fan Club,Imran M Jemal,2006,175
...,...,...,...,...,...
352660,799023333537673,Denmark for Bernie Sanders,These ads ran without a disclaimer,≤100,2
368255,427298174139138,Sedgwick County for Bernie Sanders,These ads ran without a disclaimer,≤100,1
372910,142457072843777,Rogers Park For Bernie Sanders,These ads ran without a disclaimer,≤100,1
373310,158942027817875,Filipino Americans for Bernie Sanders,These ads ran without a disclaimer,≤100,1


In [8]:
#Extracting the data to the df with all the fetched new page_ids
df_missing["page_id"] = df_missing.apply(lambda x: get_page_id(df_all_ad_pages, first_name=x['first_name'], last_name=x['last_name']),axis=1)

In [9]:
#Filtering by just one value
def get_page_id1(data: pd.DataFrame, name: str):
    
    match = data[data["Page Name"].str.contains(name, na=False)]
    if len(match) != 0:
        return match.iloc[0][0]
    else:
        return "no match"


In [11]:
#Now Cleaning the remaining with only last-name match/search
df_missing['page_id'] = df_missing.apply(lambda x: get_page_id1(df_all_ad_pages, name=x['last_name']) if x['page_id'] == 'no match' else x['page_id'], axis=1)

In [12]:
print(df_missing[df_missing.page_id == 'no match'].count()[0]) #value of no matches left
len(df_missing) #total length

7


181

# Quality Control and Cleaning Faulty Values

In [13]:
def quality_control_page_id(df, df_transparency_report):
    df_without_no_match = df[df.page_id != 'no match']
    for id in df_without_no_match.page_id:
        df_temp = df_transparency_report[df_transparency_report['Page ID'] == id]
        print("Page name transparency report: " + df_temp['Page Name'].unique())
        df_temp2 = df_without_no_match[df_without_no_match['page_id'] == id]
        print("Missing list: df_temp2: "  + df_temp2['full_name'])



In [ ]:
quality_control_page_id(df_missing, df_all_ad_pages)

['Page name transparency report: Maria Cantwell']
0    Missing list: df_temp2: Maria Cantwell
Name: full_name, dtype: object
['Page name transparency report: Roger Wicker']
6    Missing list: df_temp2: Roger F. Wicker
Name: full_name, dtype: object
['Page name transparency report: Susan Collins']
7    Missing list: df_temp2: Susan M. Collins
Name: full_name, dtype: object
['Page name transparency report: Mark Warner VA']
11    Missing list: df_temp2: Mark R. Warner
Name: full_name, dtype: object
['Page name transparency report: Kirsten Gillibrand']
12    Missing list: df_temp2: Kirsten E. Gillibrand
Name: full_name, dtype: object
['Page name transparency report: Joe Manchin']
14    Missing list: df_temp2: Joe Manchin, III
Name: full_name, dtype: object
['Page name transparency report: Robert Aderholt for Congress']
15    Missing list: df_temp2: Robert B. Aderholt
Name: full_name, dtype: object
['Page name transparency report: Michael Bennet']
16    Missing list: df_temp2: Michael F. Be

In [ ]:
#Template for the filtering method

#find the replacement value in the transparency report - looked up with parsing the correct facebook page name
replacement = df_all_ad_pages[df_all_ad_pages['Page Name'] == 'Bob Menendez']['Page ID'].iloc[0] #Have to add the iloc[0] to only get the specific value and the whole data series
row_index = df_missing[df_missing['full_name'] == 'Robert Menendez'].page_id.index[0]

type(replacement)
replacement

#column number for page_id
df_missing.columns[11]
df_missing.iat[row_index, 11] = replacement

row_index

4

In [14]:
def replace_false_pages(dict, df_missing, df_transparency_report):
    for key, value in dict.items():
        print(key, value)
        row_index = df_missing[df_missing['full_name'] == key].page_id.index[0]    
        print(row_index)
        if value == 'no ad account':
            replacement = 'no ad account'
        else:
            replacement = df_transparency_report[df_transparency_report['Page Name'] == value]['Page ID'].iloc[0] #Have to add the iloc[0] to only get the specific value and the whole data series
        
        df_missing.iat[row_index, 11] = replacement



In [ ]:
for key, value in dict_of_faulty_pid.items():
    print(key + value) 

Henry C. "Hank" Johnson, Jr.Congressman Hank Johnson
John H. RutherfordNo Ad Account
Robert MenendezBob Menendez


In [34]:
#Finding the faulty fetches:
dict_of_faulty_pid = {
        'Henry C. "Hank" Johnson, Jr.': 'Congressman Hank Johnson',
        'John H. Rutherford':'no ad account',
        'Robert Menendez': 'Bob Menendez',
        'Benjamin L. Cardin': 'no ad account',
        'H. Morgan Griffith': 'Morgan Griffith for Congress',
        'Eddie Bernice Johnson': 'no ad account',
        'Gregory W. Meeks': 'no ad account',
        'Jerry Moran':'no ad account',
        'Harold Rogers':'Hal Rogers',
        'Donald M. Payne, Jr.': 'no ad account',
        'Robin L. Kelly': 'no ad account',
        'Donald S. Beyer, Jr.': 'no ad account',
        'J. Luis Correa':'Rep. Lou Correa',
        'A. Drew Ferguson IV':'Congressman Drew Ferguson',
        'W. Gregory Steube':'Congressman Greg Steube',
        'C. Scott Franklin':'Congressman Scott Franklin',
        'Bradley Scott Schneider':'Congressman Brad Schneider',
        'Scott DesJarlais': 'no ad account',
        'Tom O’Halleran':'Rep. Tom O\'Halleran',
        'Louie Gohmert': 'no ad account',
        'John Hoeven':'no ad account',
        'Lisa Murkowski':'no ad account',
        'Aumua Amata Coleman Radewagen':'no ad account',
        'Jenniffer González-Colón':'no ad account'
        }


## Documentation of filtering process:

The page_ids for the ad campaign pages of the politicians are not accessible anywhere publically. Therefore, we have used the transparency report of all ad pages to fetch the page ids of the US Congress members. This is done in three steps. First we filter by "Full_name". This was highly efficient as most politicians used their full name for their page name. 

The second phase of filtering then occuring by a regex statement with first and last name. This gets us down only missing 66 politicians.

For the last 66 polticians we use the lastname alone for the filtering. 


There are several cases when we have tried to filter out pages that it resulted in a false data_set, some examples are presented below.
I had to go through full set of 180 manually to filter these cases out and clean the data. This was gone by cross checking and finding their facebook page, then trying to find their facebook account in the transparency report (both in the dataframe version and the website interactive page)

### Examples: 

["Page name transparency report: Mark Amodei Isn't Fighting for Us"]
88    Missing list: df_temp2: Mark E. Amodei

['Page name transparency report: Sharon Whitehurst-Payne for San Diego School Board']
90    Missing list: df_temp2: Donald M. Payne, Jr.

['Page name transparency report: Captain Mark Kelly']
102    Missing list: df_temp2: Robin L. Kelly

While Robin Kelly has a facebook account with many followers, she does not have an ad account.
https://www.facebook.com/reprobinkelly

['Page name transparency report: Jay Schneiderman for Southampton Town Supervisor']
123    Missing list: df_temp2: Bradley Scott Schneider


['Page name transparency report: Help Bring Justice To Robert D. Correale, Esq., his former law firm, Maynard & Truland']
127    Missing list: df_temp2: J. Luis Correa

These have been cleaned out by going through the dataset manually.

### Could be good to add multiple pages for some of them? Or at least we have mention this in our limitations
additional - extra pages for politicians:
Drew Ferguson
Greg Steube

### all polticians have facebook pages not all have ad accounts 

--> a lot have multiple pages (both 'senator bob connison' and 'bob connison')


In [39]:
replace_false_pages(dict_of_faulty_pid, df_missing, df_all_ad_pages)

Henry C. "Hank" Johnson, Jr. Congressman Hank Johnson
41
John H. Rutherford no ad account
129
Robert Menendez Bob Menendez
4
Benjamin L. Cardin no ad account
1
H. Morgan Griffith Morgan Griffith for Congress
34
Eddie Bernice Johnson no ad account
40
Gregory W. Meeks no ad account
54
Jerry Moran no ad account
55
Harold Rogers Hal Rogers
66
Donald M. Payne, Jr. no ad account
90
Robin L. Kelly no ad account
102
Donald S. Beyer, Jr. no ad account
117
J. Luis Correa Rep. Lou Correa
127
A. Drew Ferguson IV Congressman Drew Ferguson
134
W. Gregory Steube Congressman Greg Steube
142
C. Scott Franklin Congressman Scott Franklin
166
Bradley Scott Schneider Congressman Brad Schneider
123
Scott DesJarlais no ad account
27
Tom O’Halleran Rep. Tom O'Halleran
124
Louie Gohmert no ad account
32
John Hoeven no ad account
38
Lisa Murkowski no ad account
56
Aumua Amata Coleman Radewagen no ad account
121
Jenniffer González-Colón no ad account
138


In [31]:

df_all_ad_pages[df_all_ad_pages['Page Name'].str.contains('González-Colón', na=False)].head(20)

,Page ID,Page Name,Disclaimer,Amount Spent (USD),Number of Ads in Library


In [36]:
df_missing[df_missing.page_id.str.contains("no match", na=False)]

,Unnamed: 0,first_name,last_name,full_name,type,state,district,party,facebook,page_name,identifier,page_id,already_fetched


In [37]:
df_missing

,Unnamed: 0,first_name,last_name,full_name,type,state,district,party,facebook,page_name,identifier,page_id,already_fetched
0,1,Maria,Cantwell,Maria Cantwell,sen,WA,NaN,Democrat,senatorcantwell,Senator Maria Cantwell,100044609161538,9351652533,missing
1,2,Benjamin,Cardin,Benjamin L. Cardin,sen,MD,NaN,Democrat,senatorbencardin,Senator Ben Cardin,100044326544838,no ad account,missing
2,3,Thomas,Carper,Thomas R. Carper,sen,DE,NaN,Democrat,tomcarper,Senator Tom Carper,100044593172202,165571920202215,missing
3,4,Robert,Casey,"Robert P. Casey, Jr.",sen,PA,NaN,Democrat,SenatorBobCasey,U.S. Senator Bob Casey,100044143479624,6781166087,missing
4,7,Robert,Menendez,Robert Menendez,sen,NJ,NaN,Democrat,senatormenendez,Senator Bob Menendez,100044150273701,6520632191,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,510,Andrew,Garbarino,Andrew R. Garbarino,rep,NY,2.0,Republican,NaN,NaN,0,114371076772585,missing
177,515,Stephanie,Bice,Stephanie I. Bice,rep,OK,5.0,Republican,NaN,NaN,0,327202261326682,missing
178,525,Blake,Moore,Blake D. Moore,rep,UT,1.0,Republican,NaN,NaN,0,103147688330160,missing
179,532,Raphael,Warnock,Raphael G. Warnock,sen,GA,NaN,Democrat,NaN,NaN,0,109111900634787,missing


In [38]:
df_missing.to_csv(path_or_buf='/home/gustavgyrst/Desktop/Research Project/political-ad-api/src/data_sets/found_missing_pid.csv')

In [ ]:
# #df.loc[df['B'].isin(['one','three'])])

# # Two ways to attack this filtering of the data:
# # data.loc[data['page_name'].isin(['Boost the News'])]
# # data.loc[data['page_name'] == 'Boost the News']

# # data['page_name'].str.contains('Boost', 'Climate').any()

# # data['page_name'].filter(regex='Boost')

# # @USCongressmanFilemonVela etc.
# set_of_pol_pgnames = {'USCongressmanFilemonVela', 'reppaultonko', 'SenatorThomTillis', 'DonaldNorcrossNJ', '153423912663', 'repalgreen', 'RepDonBeyer', 'RepRogerWilliams', 'CongressmanSethMoulton', 'replizcheney', 'RepRoybalAllard', 'mitchmcconnell', 'SenatorRandPaul', 'RepSteveScalise', 'RepMoBrooks', 'CongressmanAlexMooney', 'RepJackBergman', 'RepWesterman', 'RepMcEachin', 'CongresswomanAnnieKuster', 'RepFrankLucas', 'CongressmanTedDeutch', 'GwenSMoore', 'RepTedLieu', 'RepGregMurphy', 'senatordebfischer', 'RepBobGibbs', 'AndyHarrisMD', 'WhipHoyer', 'CongressmanMichaelF.Q.SanNicolas', 'congressmangkbutterfield', 'SenatorTimScott', 'RepJohnCurtis', 'CongressmanNadler', '50375006903', 'RepBarbaraLee', 'jeffmerkley', 'JoeManchinIII', 'RepBost', 'CongressmanBuchanan', 'RepJeffDuncan', 'Congressman.Keating', 'RepSewell', 'RepRickCrawford', 'RepChrisStewart', 'zoelofgren', 'CongressmanWarrenDavidson', 'drnealdunnfl2', 'RepTomSuozzi', 'SenatorLankford', 'RepCheri', 'RepKinzinger', 'marshablackburn', 'SenBrianSchatz', 'RepDonBacon', 'SenatorHassan', 'senshelley', 'senatorcantwell', 'BlaineLuetkemeyer', '113303673339', 'RepJohnLarson', 'senatortester', 'RepRouzer', 'RobertAderholt', 'reptrey', 'jameseclyburn', 'RepRichHudson', 'RickScottSenOffice', 'SenGaryPeters', 'repschrader', 'RepKweisiMfume', 'CongressmanSteveCohen', 'senatortammybaldwin', 'RepAdamSchiff', 'RepBobbyScott', 'RepEliseStefanik', 'senatorelizabethwarren', 'CongressmanMattCartwright', 'RepAmiBera', 'CongressmanKevinMcCarthy', 'RepAnthonyBrown', 'repgaramendi', 'CongressmanDavidCicilline', 'RepJimBanks', 'JodeyArrington', 'repgracemeng', 'bill.posey15', 'senatorchriscoons', 'RepTedBudd', 'RepBrianHiggins', 'RepMarkTakano', 'CongressmanBennieGThompson', 'emanuelcleaverii', 'RepMarkDeSaulnier', 'RepHuffman', 'SenatorTomCotton', 'repscottperry', 'RepresentativeValDemings', 'senatorjimrisch', 'ScottDesJarlaisTN04', 'RepLarryBucshon', 'repchuck', 'RepWalberg', 'senbennetco', 'RepLoisFrankel', 'reptomrice', 'CongressmanClayHiggins', 'SenatorJohnHoeven', 'RepBonnie', 'RepHuizenga', 'CongressmanMikeDRogers', 'RepJoyceBeatty', 'SenatorSasse', 'CongressmanMarcVeasey', 'DianaDeGette', 'chrismurphyct', 'gregorymeeksny05', 'CongressmanDKDavis', 'CongresswomanRosaDeLauro', 'senatorbencardin', 'SenatorMikeRounds', 'RepAustinScott', 'mcmorrisrodgers', '152569121550', 'RepJuliaBrownley', 'USSenTinaSmith', 'repmarkpocan', 'timryan', 'herrerabeutler', 'sen.johncornyn', 'repstephenlynch', 'RepJoshG', 'RepJuanVargas', 'CongressmanGuthrie', 'repfrankpallone', '81058818750', 'tomcarper', 'repgosar', 'senjoniernst', 'CongressmanBillFoster', 'repcardenas', 'SenatorKaine', 'RepJimMcGovern', 'RepConorLamb', 'RepAdamSmith', 'RepDavidKustoff', 'RepDebbieLesko', 'RepKathleenRice', 'johnbarrasso', 'SenJackReed', 'RepJimmyPanetta', 'RepSteveChabot', 'reprobinkelly', 'pascrell', 'CongressmanKevinCramer', 'CongressmanGarretGraves', 'JohnKennedyLouisiana', 'senatorfeinstein', 'RepMullin', 'RepRubenGallego', 'RepScottPeters', 'joecourtney', 'RepSmucker', 'TomColeOK04', 'CongresswomanLindaSanchez', 'CongressmanDougLamborn', 'RepMikeJohnson', 'SenatorRichardBurr', 'repbrianmast', 'repjimjordan', 'RepMikeTurner', 'DonaldPayneJr', 'SenatorPatrickLeahy', 'RepTomReed', 'CongressmanBoyle', 'SenatorCortezMasto', 'ChelliePingree', 'RepRoKhanna', 'senatormikelee', 'CongressmanHalRogers', 'SenatorMarcoRubio', 'jerrymoran', 'chrisvanhollen', 'RepDelBene', 'CongressmanJimHimes', 'RepRobWittman', 'CongressmanRickAllen', 'jiminhofe', 'RepEspaillat', 'RepDebbieDingell', 'RepBradWenstrup', 'RepKarenBass', 'TXRandy14', 'RepFredUpton', 'SenatorTedCruz', 'RepresentativeMarcyKaptur', 'RepMikeGallagher', 'RepDonYoung', '395759603917487', 'susancollins', 'RepBillJohnson', '118514606128', 'RepRonEstes', 'RepSeanMaloney', 'RepMcKinley', 'SenatorAngusSKingJr', 'senronjohnson', 'repdavidschweikert', 'RepMikeThompson', 'RepMoolenaar', 'RepAndyBiggs', 'RepStephMurphy', 'sanfordbishop', 'RepSarbanes', 'CongressmanGaryPalmer', 'RepSteveWomack', 'RepLeeZeldin', 'RepDanKildee', 'JoeWilson', '81125319109', 'congressmancomer', '96007744606', 'GusBilirakis', '115356957005', 'RepDwightEvans', 'RepPerlmutter', 'RepJackieWalorski', 'CongressmanBradSchneider', 'CongressmanFredKeller', 'aumuaamata', 'CongressmanJodyHice', 'reprichardneal', 'replahood', 'SenatorDurbin', 'RepGrothman', 'RepLouCorrea', 'SenatorWhitehouse', 'RepLaMalfa', 'CongressmanMcHenry', 'RepVirginiaFoxx', 'CongressmanGerryConnolly', 'SenDanSullivan', 'RepAndyBarr', 'repbrianfitzpatrick', 'mdiazbalart', 'SenatorBlunt', 'CongresswomanTitus', 'stevenpalazzo', 'USSenatorLindseyGraham', 'RepDrewFerguson', 'mikecrapo', 'SenatorStabenow', 'DutchRupp', 'lloyddoggett', 'CongressmanEricSwalwell', 'judgecarter', 'RepJimCosta', 'RepAnnaEshoo', 'RepJackyRosen', 'CongressmanRaulRuizMD', 'senatorhirono', 'RepThomasMassie', 'congresswomanbonamici', 'SenLisaMurkowski', 'RepWilson', 'RepNormaTorres', 'RepJohnKatko', 'RepGraceNapolitano', 'JohnBoozman', 'RepHakeemJeffries', 'RepJudyChu', 'RepDaveJoyce', 'MartinHeinrich', '191056827594903', 'reploudermilk', 'CongressmanMattGaetz', 'jefffortenberry', 'RepRickLarsen', 'SenBlumenthal', 'RepAnnWagner', 'michaeltmccaul', 'RepRodneyDavis', 'RichardShelby', 'RepChrisSmith', 'doris.matsui', 'RepRutherfordFL', 'repsaludcarbajal', 'PeterWelch', 'CongressmanAndreCarson', 'NancyPelosi', 'SenDuckworth', 'CongressmanGT', 'congressmanraja', 'CongresswomanNorton', 'senatormenendez', 'JimCooper', 'MaxineWaters', 'SteveDainesMT', 'USCongressmanVicenteGonzalez', 'RepCarolynMaloney', 'CongressmanDarrenSoto', 'senschumer', 'repstaceyplaskett', 'CongresswomanAdams', 'Rep.Billy.Long', 'congressmanbuddycarter', 'SenatorShaheen', 'USRepKathyCastor', 'boblatta', 'CongresswomanSinema', 'RepJennifferGonzalezColon', 'repbettymccollum', 'repyvettedclarke', 'RepCharlieCrist', 'usrepmikedoyle', 'senatortoomey', 'RepJayapal', '326420614138023', 'RepLowenthal', 'SenatorSherrodBrown', 'jerrymcnerney', 'repjasonsmith', 'CongresswomanEBJtx30', 'CongresswomanSheilaJacksonLee', '70063393423', '63158229861', 'RepWebster', '214258646163', 'JackieSpeier', 'kevinbrady', 'senrobportman', 'Rep.BluntRochester', 'RepBrianBabin', 'repraskin', 'reptomemmer', 'repdanbishop', 'repmikequigley', 'MarkRWarner', 'EdJMarkey', 'senatorsanders', 'repohalleran', 'grassley', 'RepRalphNorman', 'RepDWS', 'RepMorganGriffith', 'MarkAmodeiNV2', 'RepJimmyGomez', 'Rep.Grijalva', 'repronkind', 'janschakowsky', 'SenKirstenGillibrand', 'repkenbuck', 'reppeteaguilar', 'RepDavidEPrice', 'SenatorToddYoung', 'CongressmanJimLangevin', 'RepFrenchHill', 'CongresswomanClark', 'SenatorBobCasey', 'Congresswoman.Hartzler', 'derek.kilmer', 'reptrentkelly', 'SenMarkKelly', 'congressmanbobbyrush', 'CongresswomanBarragan', 'RepKayGranger', 'michaelcburgess', 'RepPeterDeFazio', 'SenatorWicker', 'RepMikeGarcia', 'RepAlLawsonJr', 'RepNewhouse', '8037068318'}

# set_of_pol_full_names = {'Brendan F. Boyle', 'Andy Kim', 'Lauren Underwood', 'Thomas P. Tiffany', 'Rand Paul', 'Mike Rogers', 'Salud O. Carbajal', 'Gary C. Peters', 'John Katko', 'Greg Pence', 'Eddie Bernice Johnson', 'Jim Costa', 'Aumua Amata Coleman Radewagen', 'Suzanne Bonamici', 'Tony Gonzales', 'Raja Krishnamoorthi', 'David B. McKinley', 'Stephanie I. Bice', 'Victoria Spartz', 'A. Drew Ferguson IV', 'Mary E. Miller', 'Lucille Roybal-Allard', 'Kirsten E. Gillibrand', 'Jeff Fortenberry', 'Darren Soto', 'John B. Larson', 'Troy Balderson', 'James Comer', 'Ken Buck', 'Russ Fulcher', 'Eric A. "Rick" Crawford', 'Brian Schatz', 'Doug Lamborn', 'Henry C. "Hank" Johnson, Jr.', 'Tom Emmer', 'Sam Graves', 'Alexandria Ocasio-Cortez', 'Richard Burr', 'Patty Murray', 'Ed Perlmutter', 'Charlie Crist', 'Kevin Brady', 'Lisa C. McClain', 'Cori Bush', 'Bill Cassidy', 'Mike Garcia', 'Ben Sasse', 'Tina Smith', 'Robert B. Aderholt', 'Mark DeSaulnier', 'Nicole Malliotakis', 'Al Green', 'Mariannette Miller-Meeks', 'Thomas R. Suozzi', 'Christopher H. Smith', 'Debbie Wasserman Schultz', 'Trey Hollingsworth', 'Mike Levin', 'Troy E. Nehls', 'Zoe Lofgren', 'Debbie Lesko', 'Mo Brooks', 'Matthew M. Rosendale', 'Mark E. Amodei', 'Rick W. Allen', 'Bill Hagerty', 'Donald Norcross', 'Marie Newman', 'Chris Van Hollen', 'Robert J. Wittman', 'Grace Meng', 'Nikema Williams', 'Dean Phillips', 'Pat Fallon', 'Eric Swalwell', 'Earl L. "Buddy" Carter', 'Kurt Schrader', 'Dan Bishop', 'Sherrod Brown', 'Ted Lieu', 'Jason Crow', 'Judy Chu', 'Cathy McMorris Rodgers', 'Anthony Gonzalez', 'Kaiali’i Kahele', 'Cynthia Axne', 'Jody B. Hice', 'Jack Bergman', 'Nancy Mace', 'Kevin Hern', 'Richard Blumenthal', 'Kathy E. Manning', 'David Kustoff', 'Greg Stanton', 'Richard C. Shelby', 'Michael T. McCaul', 'Tim Walberg', 'David N. Cicilline', 'Mark R. Warner', 'Paul Tonko', 'Val Butler Demings', 'Mark E. Green', 'Lois Frankel', 'Steve Daines', 'Tracey Mann', 'Guy Reschenthaler', 'Kat Cammack', 'Roy Blunt', 'André Carson', 'Alexander Mooney', 'Adrian Smith', 'Bob Gibbs', 'Pete Aguilar', 'Richard E. Neal', 'Ann Kirkpatrick', 'Fred Keller', 'Scott H. Peters', 'Andy Barr', 'Mark Kelly', 'Cindy Hyde-Smith', 'Joaquin Castro', 'Joe Wilson', 'Jack Reed', 'Lance Gooden', 'Thomas Massie', 'David Schweikert', 'Bill Johnson', 'John A. Yarmuth', 'Scott Fitzgerald', 'Roger Williams', 'Andy Biggs', 'Tom Malinowski', 'Joni Ernst', 'Angus S. King, Jr.', 'Patrick J. Leahy', 'C. A. Dutch Ruppersberger', 'John R. Moolenaar', 'Virginia Foxx', 'Tom Rice', 'John Thune', 'Barry Moore', 'Devin Nunes', 'Rashida Tlaib', 'Marc A. Veasey', 'Joe Neguse', 'Madison Cawthorn', 'Mike Bost', 'Pramila Jayapal', 'Bob Good', 'Jesús G. "Chuy" García', 'Larry Bucshon', 'Jared Huffman', 'Chris Jacobs', 'Mary Gay Scanlon', 'James E. Risch', 'Nancy Pelosi', 'Gregory W. Meeks', 'John W. Hickenlooper', 'Rick Scott', 'Jay Obernolte', 'Charles J. "Chuck" Fleischmann', 'Katherine M. Clark', 'Lizzie Fletcher', 'Jared F. Golden', 'Derek Kilmer', 'Tim Scott', 'Richard J. Durbin', 'Jim Cooper', 'Ann Wagner', 'Mondaire Jones', 'Michael C. Burgess', 'Cheri Bustos', 'August Pfluger', 'Sean Patrick Maloney', 'Edward J. Markey', 'Bernard Sanders', 'Ron Estes', 'Amy Klobuchar', 'Haley M. Stevens', 'Hakeem S. Jeffries', 'Dan Crenshaw', 'Tim Burchett', 'Tim Ryan', 'Michael K. Simpson', 'Diana DeGette', 'Todd Young', 'Melanie A. Stansbury', 'Lee M. Zeldin', 'Filemon Vela', 'Elissa Slotkin', 'Michael F. Bennet', 'Terri Sewell', 'Tom Cotton', 'Andrew R. Garbarino', 'Andrew S. Clyde', 'Markwayne Mullin', 'John Rose', 'James R. Langevin', 'Josh Harder', 'Burgess Owens', 'Stephen F. Lynch', 'Jake LaTurner', 'Rick Larsen', 'Jon Ossoff', 'Matt Cartwright', 'Andy Harris', 'Blaine Luetkemeyer', 'Michelle Steel', 'Garret Graves', 'Mikie Sherrill', 'Mike Quigley', 'Gregory F. Murphy', 'Richard Hudson', 'Liz Cheney', 'John P. Sarbanes', 'Elizabeth Warren', 'G. K. Butterfield', 'Ami Bera', 'Brad R. Wenstrup', 'James A. Himes', 'Neal P. Dunn', 'Byron Donalds', 'John Kennedy', 'Chellie Pingree', 'Chris Stewart', 'Bill Pascrell, Jr.', 'Brett Guthrie', 'Marsha Blackburn', 'Louie Gohmert', 'Lisa Blunt Rochester', 'Jenniffer González-Colón', 'Ken Calvert', 'Bradley Scott Schneider', 'Dan Sullivan', 'Don Bacon', 'Sean Casten', 'Brian K. Fitzpatrick', 'Kathleen M. Rice', 'Susie Lee', 'Yvette D. Clarke', 'Nydia M. Velázquez', 'Julia Brownley', 'Ayanna Pressley', 'William R. Timmons IV', 'Shelley Moore Capito', 'Mike Lee', 'Jahana Hayes', 'Michael F. Q. San Nicolas', 'Maria Elvira Salazar', 'Brian Babin', 'Chris Pappas', 'Carolyn Bourdeaux', 'Rob Portman', 'Maxine Waters', 'Jaime Herrera Beutler', 'Dianne Feinstein', 'Tom Cole', 'Anthony Brown', 'Steve Chabot', 'Ben Ray Luján', 'Thom Tillis', 'Mike Johnson', 'Karen Bass', 'Thomas R. Carper', 'Beth Van Duyne', 'Barry Loudermilk', 'Gus M. Bilirakis', 'Martin Heinrich', 'Donald S. Beyer, Jr.', 'Jennifer Wexton', 'Pete Sessions', 'Teresa Leger Fernandez', 'Gerald E. Connolly', 'Mike Kelly', 'Marco Rubio', 'Elaine G. Luria', 'Ted Budd', 'Rodney Davis', 'Adam Kinzinger', 'Yvette Herrell', 'Doug LaMalfa', 'Lucy McBath', 'Adam Smith', 'Lisa Murkowski', 'H. Morgan Griffith', 'Brenda L. Lawrence', 'Patrick T. McHenry', 'Jason Smith', 'Michael Guest', 'Ron Kind', 'Kyrsten Sinema', 'Jake Auchincloss', 'Bill Posey', 'Ralph Norman', 'Robert C. "Bobby" Scott', 'Michael Cloud', 'Jackie Speier', 'Margaret Wood Hassan', 'Tommy Tuberville', 'Gregorio Kilili Camacho Sablan', 'Dwight Evans', 'Ed Case', 'Josh Gottheimer', 'Lori Trahan', 'Lloyd Smucker', 'Raúl M. Grijalva', 'Jacky Rosen', 'Ron Wyden', 'Jimmy Gomez', 'David E. Price', 'J. Hill', 'Joe Manchin, III', 'Gwen Moore', 'Tom O’Halleran', 'Bobby L. Rush', 'Catherine Cortez Masto', 'Kim Schrier', 'Roger F. Wicker', 'Joseph D. Morelle', 'Dusty Johnson', 'Harold Rogers', 'Peter A. DeFazio', 'Trent Kelly', 'Christopher A. Coons', 'Lauren Boebert', 'Maria Cantwell', 'W. Gregory Steube', 'Patrick J. Toomey', 'Donald M. Payne, Jr.', 'Benjamin L. Cardin', 'Henry Cuellar', 'Steny H. Hoyer', 'Jim Banks', 'Josh Hawley', 'Carol D. Miller', 'Kathy Castor', 'Frederica S. Wilson', 'Robin L. Kelly', 'Tammy Baldwin', 'Paul A. Gosar', 'Elise M. Stefanik', 'James E. Clyburn', 'Robert P. Casey, Jr.', 'Daniel Webster', 'Cory A. Booker', 'Debbie Stabenow', 'Matt Gaetz', 'Steven Horsford', 'Ben Cline', 'James R. Baird', 'David G. Valadao', 'Lindsey Graham', 'Stephanie N. Murphy', 'Van Taylor', 'Vicente Gonzalez', 'Claudia Tenney', 'Don Young', 'David Rouzer', 'Peter Meijer', 'Nanette Diaz Barragán', 'Randy K. Weber, Sr.', 'Austin Scott', 'Tim Kaine', 'Tony Cárdenas', 'Joe Courtney', 'Tom Reed', 'Kay Granger', 'Ron Johnson', 'Warren Davidson', 'John H. Rutherford', 'Albio Sires', 'Ashley Hinson', 'Raphael G. Warnock', 'Fred Upton', 'John Barrasso', 'Ronny Jackson', 'Charles E. Schumer', 'Troy A. Carter', 'Seth Moulton', 'Tom McClintock', 'J. Luis Correa', 'Angie Craig', 'Frank D. Lucas', 'Roger Marshall', 'Mitch McConnell', 'Jackie Walorski', 'Joyce Beatty', 'Lloyd Doggett', 'David J. Trone', 'Brad Sherman', 'John R. Carter', 'Jamaal Bowman', 'Steve Womack', 'Jerry L. Carl', 'Michelle Fischbach', 'Mark Pocan', 'Glenn Grothman', 'Raul Ruiz', 'Sylvia R. Garcia', 'Jim Jordan', 'Mitt Romney', 'Deb Fischer', 'Peter Welch', 'Darrell Issa', 'James P. McGovern', 'Rosa L. DeLauro', 'Janice D. Schakowsky', 'Dina Titus', 'Sharice Davids', 'Mario Diaz-Balart', 'Blake D. Moore', 'Daniel T. Kildee', 'Jerry McNerney', 'Ritchie Torres', 'Adriano Espaillat', 'Andy Levin', 'Betty McCollum', 'Frank Pallone, Jr.', 'Randy Feenstra', 'John R. Curtis', 'Ann Kuster', 'Jim Hagedorn', 'Robert Menendez', 'Cynthia M. Lummis', 'Bennie G. Thompson', 'Jon Tester', 'Marcy Kaptur', 'Alma S. Adams', 'Suzan K. DelBene', 'Brian J. Mast', 'Jodey C. Arrington', 'James M. Inhofe', 'Conor Lamb', 'Madeleine Dean', 'Marjorie Taylor Greene', 'Steve Scalise', 'Sheila Jackson Lee', 'Daniel Meuser', 'Sanford D. Bishop, Jr.', 'Jeanne Shaheen', 'Ruben Gallego', 'A. Donald McEachin', 'Bill Huizenga', 'Doris O. Matsui', 'Bill Foster', 'Gary J. Palmer', 'Stacey E. Plaskett', 'Darin LaHood', 'Abigail Davis Spanberger', 'John Garamendi', 'Adam B. Schiff', 'David P. Joyce', 'Al Lawson, Jr.', 'Chrissy Houlahan', 'Mike Crapo', 'Glenn Thompson', 'Frank J. Mrvan', 'Jamie Raskin', 'Dan Newhouse', 'John Joyce', 'Susan Wild', 'Carlos A. Gimenez', 'Alex Padilla', 'Eleanor Holmes Norton', 'Alan S. Lowenthal', 'John Hoeven', 'Susan M. Collins', 'Mike Thompson', 'Pete Stauber', 'John Cornyn', 'Scott Perry', 'David Scott', 'Robert E. Latta', 'Jeff Duncan', 'Michael Waltz', 'Jimmy Panetta', 'Jefferson Van Drew', 'Veronica Escobar', 'Ilhan Omar', 'Anna G. Eshoo', 'Scott DesJarlais', 'Christopher Murphy', 'Chuck Grassley', 'William R. Keating', 'Debbie Dingell', 'Vern Buchanan', 'Chip Roy', 'Kelly Armstrong', 'Kevin McCarthy', 'Ro Khanna', 'Cliff Bentz', 'Diana Harshbarger', 'Brian Higgins', 'Norma J. Torres', 'Jerry Moran', 'Bryan Steil', 'Mike Braun', 'Carolyn B. Maloney', 'Earl Blumenauer', 'Deborah K. Ross', 'Mazie K. Hirono', 'Young Kim', 'Sara Jacobs', 'Barbara Lee', 'Tammy Duckworth', 'Julia Letlow', 'John Boozman', 'Bruce Westerman', 'Grace F. Napolitano', 'Linda T. Sánchez', 'Mark Takano', 'Katie Porter', 'Theodore E. Deutch', 'Mike Gallagher', 'Michael R. Turner', 'Ted Cruz', 'Sheldon Whitehouse', 'Bonnie Watson Coleman', 'James Lankford', 'Colin Z. Allred', 'Kevin Cramer', 'C. Scott Franklin', 'Steven M. Palazzo', 'Antonio Delgado', 'Jeff Merkley', 'Emanuel Cleaver', 'Danny K. Davis', 'Kweisi Mfume', 'Juan Vargas', 'Marilyn Strickland', 'Steve Cohen', 'Clay Higgins', 'Vicky Hartzler', 'Michael F. Doyle', 'Billy Long', 'Jerrold Nadler', 'Mike Rounds'}

# white_house = {'Joe Biden', 'Kamala Harris', 'Barack Obama', 'Donald J. Trump', 'Mike Pence'}

# combined = white_house.union(set_of_pol_full_names)

# #To check for substring
# data.loc[data['page_name'].str.contains('Congressman')]

# data.loc[data['page_name'].str.contains('Congresswoman')]

# data.loc[data['page_name'].str.contains('Senator')]
# #Results with this

# data.loc[data['page_name'].str.contains('Governor')]
# #Results with this

# #pol_fbpage_names = get_pol_fbpage_names()

# df_by_pol = data.loc[data['page_name'].isin(combined)]

# df_by_pol.groupby('page_name').count()